In [1]:
import pandas as pd
pd.set_option('display.max_rows', 20)

### Notes:

The github repo https://github.com/LauraRichter/PyConZA_2017 contains:

* This noteook
* An annotated version of this notebook
* A notebook to create simulated example data pycon_example_data.csv

<h1><center>Getting the most out of your pandas</center></h1>
<br>
<h2><center>
    Laura Richter<br><br>
    Prodigy Finance
</center></h2>
<br>
<center>
![](ProdigyFinance_StackedLogo_small.png)
</center>

<h2><center>Pandas is the fastest-growing Python-related tag on stack overflow! *</center></h2>

<br>
<br>
<br>
<br>
<br>

<div style="text-align: right"> [*] https://stackoverflow.blog/2017/09/14/python-growing-quickly </div>


 <h1><center>
 &bull; Importing & Exporting
 &bull; Joining
 &bull; Grouping & Aggregating 
 &bull; <br>
 &bull; Transforming 
 &bull; Handing missing values 
 &bull; Boolean / conditional indexing
 &bull; <br>
 &bull; Built in maths and stats 
 &bull; Time series
 &bull; Visualisation
 &bull;
 </center></h1>

<h1><center>All things to all people</center></h1>

<br>

<h1><center>(Relatively) Low barrier to entry</center></h1>

---
<h2>Resources</h2>

Pandas and Dask from the Inside, Stephen Simmons <br>
Berlin, PyData 2017 <br>
https://github.com/stevesimmons/pydata-berlin2017-pandas-and-dask-from-the-inside <br>
https://www.youtube.com/watch?v=YGk09nK_xnM

<br>
No More Sad Pandas: Optimizing Pandas Code for Speed and Efficiency, Sofia Heisler <br>
Portland Oregon, PyCon 2017 <br>
https://github.com/sversh/pycon2017-optimizing-pandas <br>
https://www.youtube.com/watch?v=HN5d490_KKk <br>
https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6


---
<h2>Code!</h2>

<h4>First set up imports and profiling</h4>

In [2]:
import pandas as pd
import numpy as np

%load_ext memory_profiler
%load_ext line_profiler

In [3]:
print('hello pycon!')

hello pycon!


---
<h2>NumPy</h2>

<h2>&bull; Memory backend for pandas</h2>

<h2>&bull; Computational backend for pandas</h2>

---
<h3>Numpy for memory</h3>

In [4]:
import sys
gso = sys.getsizeof

In [5]:
%memit -r 1 -i 0 lst = list(range(987654))
%memit -r 1 -i 0 arr = np.arange(987654, dtype=np.int32)

peak memory: 128.96 MiB, increment: 38.19 MiB
peak memory: 132.87 MiB, increment: 3.88 MiB


In [6]:
# size in bytes / MiB:

print('List:')
print(gso(0), 'B')
print(gso([]), 'B')
print(gso([0]), 'B')
print('{:.2f} MiB'.format(gso(lst)/(2**20)))
full_size = gso(lst) + sum([gso(x) for x in lst])
print('{:.2f} MiB'.format(full_size/(2**20)))

print()
print('Numpy array:')
print('{:.2f} MiB'.format(gso(arr)/(2**20)))

List:
24 B
64 B
72 B
8.48 MiB
34.85 MiB

Numpy array:
3.77 MiB


---
<h3>Numpy for speed</h3>

In [7]:
lst = range(5555)
%timeit -n 500 sum(lst)

86.6 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [8]:
arr = np.arange(5555)
%timeit -n 500 np.sum(arr)
%timeit -n 500 arr.sum()

6.28 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
5.7 µs ± 830 ns per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [9]:
%timeit -n 500 [l+2 for l in lst]
%timeit -n 500 arr + 2

286 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
2.63 µs ± 84.7 ns per loop (mean ± std. dev. of 7 runs, 500 loops each)


---
<h2>Pandas data structures: The DataFrame</h2>

In [10]:
col1 = np.arange(6)
col2 = np.arange(4.5, 10.5, 1.0)
col3 = 'a,b,c,d,e,f'.split(',')
col4 = 'one,two,three,four,five,six'.split(',')
col5 = np.arange(45, 51)
col6 = [22.3, 98.3, 52.0, 7.6, 43.9, 10.1]
col7 = np.arange(62, 68, 1.0) 
indx = ['r'+str(i) for i in range(6)]

example_df = pd.DataFrame(col1, columns=['col1'], index=indx)
example_df['col2'] = col2
example_df['col3'] = col3
example_df['col4'] = col4
example_df['col5'] = col5
example_df['col6'] = col6
example_df['col7'] = col7

# dictionary of column colors
coldict = {
    'col1': '#c5a3ff',
    'col5': '#c5a3ff',
    'col2': '#dbffd6',
    'col6': '#dbffd6',
    'col7': '#dbffd6',
    'col3': '#ffcbc1',
    'col4': '#ffcbc1',
}

def highlight_cols(s, coldict):
    if s.name in coldict.keys():
        return ['background-color: {}'.format(coldict[s.name])] * len(s)
    return [''] * len(s)

In [11]:
example_df.style.apply(highlight_cols, coldict=coldict)

,col1,col2,col3,col4,col5,col6,col7
r0,0,4.5,a,one,45,22.3,62
r1,1,5.5,b,two,46,98.3,63
r2,2,6.5,c,three,47,52,64
r3,3,7.5,d,four,48,7.6,65
r4,4,8.5,e,five,49,43.9,66
r5,5,9.5,f,six,50,10.1,67


In [12]:
example_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, r0 to r5
Data columns (total 7 columns):
col1    6 non-null int64
col2    6 non-null float64
col3    6 non-null object
col4    6 non-null object
col5    6 non-null int64
col6    6 non-null float64
col7    6 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 544.0+ bytes


In [13]:
example_df.loc['r2', 'col1'] = np.nan;

In [14]:
# dictionary of column colors
coldict = {
    'col1': '#dbffd6',
    'col5': '#c5a3ff',
    'col2': '#dbffd6',
    'col6': '#dbffd6',
    'col7': '#dbffd6',
    'col3': '#ffcbc1',
    'col4': '#ffcbc1',
}

def highlight_cols(s, coldict):
    if s.name in coldict.keys():
        return ['background-color: {}'.format(coldict[s.name])] * len(s)
    return [''] * len(s)

In [15]:
example_df.info()
example_df.style.apply(highlight_cols, coldict=coldict)

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, r0 to r5
Data columns (total 7 columns):
col1    5 non-null float64
col2    6 non-null float64
col3    6 non-null object
col4    6 non-null object
col5    6 non-null int64
col6    6 non-null float64
col7    6 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 544.0+ bytes


,col1,col2,col3,col4,col5,col6,col7
r0,0,4.5,a,one,45,22.3,62
r1,1,5.5,b,two,46,98.3,63
r2,nan,6.5,c,three,47,52,64
r3,3,7.5,d,four,48,7.6,65
r4,4,8.5,e,five,49,43.9,66
r5,5,9.5,f,six,50,10.1,67


---
<h2>Data set: Prodigy Finance loan application data</h2>

In [16]:
!ls -lh | grep pycon_example_data.csv

-rw-r--r-- 1 laura laura 1.9M Oct  8 21:49 pycon_example_data.csv


In [17]:
%memit -r 1 -i 0 df = pd.read_csv('pycon_example_data.csv')

nrows, ncols = df.shape
print('Rows:   ', nrows)
print('Columns:', ncols)
column_names = df.columns

peak memory: 123.36 MiB, increment: 15.40 MiB
Rows:    20000
Columns: 14


In [18]:
df.head()

,average_3_month_salary,average_3_month_salary_gbp_rate,country_of_habitual_residence,default_gbp_rate,latest_step,other_assets,other_assets_gbp_rate,payback_period,pre_study_country,property_value,property_value_gbp_rate,school_type,school_university_name,total_liabilities_in_base_currency
0,1458.997,0.866,ET,0.474,2,541.529,1.267,180,IM,325.148,0.555,Engineering,New York University,18810.277
1,2177.916,1.299,CN,1.337,6,757.306,0.802,180,MK,1668.911,1.362,Business,University of Pennsylvania,11203.819
2,1445.805,0.858,TW,0.923,6,290.339,0.579,240,PH,944.007,1.168,Engineering,University of Michigan,NaN
3,1080.797,1.508,SG,0.650,8,602.398,1.482,60,KE,1339.878,1.076,Engineering,University of Oxford,18493.556
4,809.031,0.566,GA,1.199,5,617.201,0.806,240,FI,2643.022,1.603,Engineering,Georgetown University,NaN


---
<h2>DataFrame internals</h2>

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
average_3_month_salary                17626 non-null float64
average_3_month_salary_gbp_rate       20000 non-null float64
country_of_habitual_residence         19877 non-null object
default_gbp_rate                      20000 non-null float64
latest_step                           20000 non-null int64
other_assets                          18086 non-null float64
other_assets_gbp_rate                 20000 non-null float64
payback_period                        20000 non-null int64
pre_study_country                     19882 non-null object
property_value                        16939 non-null float64
property_value_gbp_rate               20000 non-null float64
school_type                           20000 non-null object
school_university_name                19376 non-null object
total_liabilities_in_base_currency    16912 non-null float64
dtypes: float64(8), int64(2), object(4)
memor

In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
average_3_month_salary                17626 non-null float64
average_3_month_salary_gbp_rate       20000 non-null float64
country_of_habitual_residence         19877 non-null object
default_gbp_rate                      20000 non-null float64
latest_step                           20000 non-null int64
other_assets                          18086 non-null float64
other_assets_gbp_rate                 20000 non-null float64
payback_period                        20000 non-null int64
pre_study_country                     19882 non-null object
property_value                        16939 non-null float64
property_value_gbp_rate               20000 non-null float64
school_type                           20000 non-null object
school_university_name                19376 non-null object
total_liabilities_in_base_currency    16912 non-null float64
dtypes: float64(8), int64(2), object(4)
memor

---
<h2>The block manager</h2>

In [21]:
df._data

BlockManager
Items: Index(['average_3_month_salary', 'average_3_month_salary_gbp_rate',
       'country_of_habitual_residence', 'default_gbp_rate', 'latest_step',
       'other_assets', 'other_assets_gbp_rate', 'payback_period',
       'pre_study_country', 'property_value', 'property_value_gbp_rate',
       'school_type', 'school_university_name',
       'total_liabilities_in_base_currency'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=20000, step=1)
FloatBlock: [0, 1, 3, 5, 6, 9, 10, 13], 8 x 20000, dtype: float64
IntBlock: slice(4, 10, 3), 2 x 20000, dtype: int64
ObjectBlock: [2, 8, 11, 12], 4 x 20000, dtype: object

In [22]:
df.memory_usage(deep=True)

Index                                      80
average_3_month_salary                 160000
average_3_month_salary_gbp_rate        160000
country_of_habitual_residence         1176893
default_gbp_rate                       160000
latest_step                            160000
other_assets                           160000
other_assets_gbp_rate                  160000
payback_period                         160000
pre_study_country                     1177046
property_value                         160000
property_value_gbp_rate                160000
school_type                           1330159
school_university_name                1532929
total_liabilities_in_base_currency     160000
dtype: int64

---
<h2>Sidebar: The category type</h2>

In [23]:
pd.set_option('display.max_rows', 12)

In [24]:
df['country_of_habitual_residence']

0        ET
1        CN
2        TW
3        SG
4        GA
5        MU
         ..
19994    KH
19995    RW
19996    KY
19997    CY
19998    JM
19999    NP
Name: country_of_habitual_residence, Length: 20000, dtype: object

In [25]:
pd.set_option('display.max_rows', 20)

In [26]:
df['school_university_name'].unique()

array(['New York University', 'University of Pennsylvania',
       'University of Michigan', 'University of Oxford',
       'Georgetown University', 'Stanford University',
       'Cornell University', 'Ramon Llull University',
       'University of British Columbia', 'Boston University',
       'University of Strathclyde', 'Harvard University',
       'University of Illinois Urbana-Champaign',
       'University of Cambridge', 'University of Navarra',
       'Dartmouth College', 'City University', 'Columbia University',
       'University of Toronto', 'University of Cape Town', nan,
       'Carnegie Mellon University', 'University of Chicago',
       'York University', 'IE University', 'Lancaster University',
       'Cranfield University', 'Babson College',
       'University of Connecticut', 'University of Southern California',
       'McGill University', 'Boston College'], dtype=object)

In [27]:
df['country_of_habitual_residence'] = df['country_of_habitual_residence'].astype('category')

In [28]:
df.memory_usage(deep=True)

Index                                      80
average_3_month_salary                 160000
average_3_month_salary_gbp_rate        160000
country_of_habitual_residence           55093
default_gbp_rate                       160000
latest_step                            160000
other_assets                           160000
other_assets_gbp_rate                  160000
payback_period                         160000
pre_study_country                     1177046
property_value                         160000
property_value_gbp_rate                160000
school_type                           1330159
school_university_name                1532929
total_liabilities_in_base_currency     160000
dtype: int64

In [29]:
pd.set_option('display.max_rows', 12)

In [30]:
df['country_of_habitual_residence']

0        ET
1        CN
2        TW
3        SG
4        GA
5        MU
         ..
19994    KH
19995    RW
19996    KY
19997    CY
19998    JM
19999    NP
Name: country_of_habitual_residence, Length: 20000, dtype: category
Categories (169, object): [AD, AE, AF, AL, ..., YE, ZA, ZM, ZW]

In [31]:
pd.set_option('display.max_rows', 20)

In [32]:
df._data

BlockManager
Items: Index(['average_3_month_salary', 'average_3_month_salary_gbp_rate',
       'country_of_habitual_residence', 'default_gbp_rate', 'latest_step',
       'other_assets', 'other_assets_gbp_rate', 'payback_period',
       'pre_study_country', 'property_value', 'property_value_gbp_rate',
       'school_type', 'school_university_name',
       'total_liabilities_in_base_currency'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=20000, step=1)
FloatBlock: [0, 1, 3, 5, 6, 9, 10, 13], 8 x 20000, dtype: float64
IntBlock: slice(4, 10, 3), 2 x 20000, dtype: int64
ObjectBlock: [8, 11, 12], 3 x 20000, dtype: object
CategoricalBlock: slice(2, 3, 1), 1 x 20000, dtype: category

---
<h1>Examples</h1>

---
<h2>Example 1: Extracting data (Simple sum)</h2>

In [33]:
%timeit -n 7 -r 200 df[['average_3_month_salary']][5000:15000]
%timeit -n 7 -r 200 df['average_3_month_salary'][5000:15000]
%timeit -n 7 -r 200 df['average_3_month_salary'].iloc[5000:15000]
%timeit -n 7 -r 200 df['average_3_month_salary'].values[5000:15000]

497 µs ± 63.1 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)
52.1 µs ± 8.42 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)
35.6 µs ± 3.2 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)
3.07 µs ± 802 ns per loop (mean ± std. dev. of 200 runs, 7 loops each)


---
<h2>Example 2: Unique string values</h2>

In [34]:
df['pre_study_country'].fillna('', inplace=True)

%timeit -n 7 -r 200 np.unique(df['pre_study_country'])
%timeit -n 7 -r 200 np.unique(df['pre_study_country'].values)
%timeit -n 7 -r 200 df['pre_study_country'].unique()

9.43 ms ± 546 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)
9.31 ms ± 594 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)
332 µs ± 16.5 µs per loop (mean ± std. dev. of 200 runs, 7 loops each)


---
<h2>Example 3: Combining columns (Simple sum)</h2>

In [35]:
%%timeit
assets = []
for i in range(0, len(df)):
    a = df['property_value'].iloc[i] + df['other_assets'].iloc[i]
    assets.append(a)

429 ms ± 5.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%%timeit
assets = []
for ii, row in df.iterrows():
    a = row['property_value'] + row['other_assets']
    assets.append(a)

1.14 s ± 7.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%timeit assets = df[['property_value', 'other_assets']].apply(lambda row: np.nansum(row), axis=1)

425 ms ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%timeit assets = df.apply(lambda row: row['property_value'] + row['other_assets'], axis=1)

360 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit assets = df[['property_value', 'other_assets']].sum()

517 µs ± 7.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
%timeit assets = np.sum(df[['property_value', 'other_assets']].values, axis=1)

403 µs ± 4.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
%timeit assets = df['property_value'] + df['other_assets']

177 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
%timeit assets = df['property_value'].values + df['other_assets'].values

23.5 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


---
<h3>Sidebar: Other numerical backends: Numexpr & Bottleneck</h3>

In [43]:
%timeit assets = df.eval('property_value + other_assets')

1.29 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [44]:
pd.set_option('compute.use_numexp', False)
pd.set_option('compute.use_bottleneck', False)
%timeit assets = df['property_value'].values + df['other_assets'].values

21.1 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [45]:
pd.set_option('compute.use_numexp', True)
pd.set_option('compute.use_bottleneck', False)
%timeit -n 1 -r 50 assets = df['property_value'].values + df['other_assets'].values

The slowest run took 5.90 times longer than the fastest. This could mean that an intermediate result is being cached.
30 µs ± 20.7 µs per loop (mean ± std. dev. of 50 runs, 1 loop each)


In [46]:
pd.set_option('compute.use_numexp', False)
pd.set_option('compute.use_bottleneck',True)
%timeit assets = df['property_value'].values + df['other_assets'].values

21.4 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
pd.set_option('compute.use_numexp', True)
pd.set_option('compute.use_bottleneck', True)
%timeit assets = df['property_value'].values + df['other_assets'].values

21.6 µs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


---
<h2>Example 4: Column operations on strings</h2>

In [48]:
%timeit df['school_university_name'].apply(lambda x: '' if pd.isnull(x) else x.upper())
%timeit df['school_university_name'].str.upper()

10.5 ms ± 55.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
The slowest run took 4.56 times longer than the fastest. This could mean that an intermediate result is being cached.
7.75 ms ± 5.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


---

In [49]:
df['school_university_name'].fillna('', inplace=True)
df['school_type'].fillna('', inplace=True)

In [50]:
%timeit a = [row['school_university_name']+'_'+row['school_type'] for ii, row in df.iterrows()]

1.34 s ± 287 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit df.apply(lambda x: x['school_university_name']+'_'+x['school_type'], axis=1)
%timeit df[['school_university_name', 'school_type']].sum(axis=1)
%timeit df['school_university_name'].str.cat(df['school_type'])
%timeit df['school_university_name']+'_'+df['school_type']

390 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
15.5 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.92 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.05 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


---
<h2>Example 5: Function of several columns</h2>

In [52]:
def net_asset_salary_ratio(prop, prop_rate, assets, assets_rate, liabilities, liabilities_rate, salary, salary_rate):
    net_position = prop*prop_rate + assets*assets_rate - liabilities*liabilities_rate
    return net_position / (salary*salary_rate)

In [53]:
%%timeit -n 1 -r 5
asset_to_salary_1 = []
for ii, row in df.iterrows():
    a = net_asset_salary_ratio(
        row['property_value'], row['property_value_gbp_rate'], row['other_assets'], row['other_assets_gbp_rate'],
        row['total_liabilities_in_base_currency'], row['default_gbp_rate'], row['average_3_month_salary'],
        row['average_3_month_salary_gbp_rate']
    )
    asset_to_salary_1.append(a)

2.11 s ± 134 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [54]:
%%timeit -n 1 -r 5
asset_to_salary = []
for i in range(0, len(df)):
    a = net_asset_salary_ratio(
        df['property_value'].iloc[i], df['property_value_gbp_rate'].iloc[i], df['other_assets'].iloc[i],
        df['other_assets_gbp_rate'].iloc[i], df['total_liabilities_in_base_currency'].iloc[i],
        df['default_gbp_rate'].iloc[i], df['average_3_month_salary'].iloc[i],
        df['average_3_month_salary_gbp_rate'].iloc[i]
    )
    asset_to_salary.append(a)

2 s ± 157 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [55]:
%%timeit -n 1 -r 5
asset_to_salary = df.apply(lambda row: net_asset_salary_ratio(
        row['property_value'], row['property_value_gbp_rate'], row['other_assets'], row['other_assets_gbp_rate'],
        row['total_liabilities_in_base_currency'], row['default_gbp_rate'], row['average_3_month_salary'],
        row['average_3_month_salary_gbp_rate']
), axis=1)

1.2 s ± 51.3 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [56]:
%%timeit -n 1 -r 50
asset_to_salary = df.eval(
    '(property_value*property_value_gbp_rate + other_assets*other_assets_gbp_rate - \
    total_liabilities_in_base_currency*default_gbp_rate) / (average_3_month_salary*average_3_month_salary_gbp_rate)',
    parser='pandas', engine='numexpr'
)

2.72 ms ± 317 µs per loop (mean ± std. dev. of 50 runs, 1 loop each)


In [57]:
%%timeit -n 1 -r 5

asset_to_salary = net_asset_salary_ratio(
    df['property_value'], df['property_value_gbp_rate'], df['other_assets'], df['other_assets_gbp_rate'],
    df['total_liabilities_in_base_currency'], df['default_gbp_rate'], df['average_3_month_salary'],
    df['average_3_month_salary_gbp_rate']
)

1.51 ms ± 591 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [58]:
%%timeit -n 1 -r 5

asset_to_salary = net_asset_salary_ratio(
    df['property_value'].values, df['property_value_gbp_rate'].values,
    df['other_assets'].values, df['other_assets_gbp_rate'].values, df['total_liabilities_in_base_currency'].values,
    df['default_gbp_rate'].values, df['average_3_month_salary'].values,
    df['average_3_month_salary_gbp_rate'].values
)

227 µs ± 75.1 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


<h2> 2s to 200 microseconds = 10E4 speed up! </h2>

---
<h2>Example 5: Line profiling</h2>

Note: I added the lprn output from my run in separate markdown blocks, as the lprun output appears in a pop up window that may not have worked during the presentation. Your output will differ, but the overall comparison between the three runs should have the same form: 20000 hits for df.apply, 1 hit for the other two, and fastest time for the last run where we take .values

In [59]:
%lprun -f net_asset_salary_ratio asset_to_salary = df.apply(lambda row: net_asset_salary_ratio(row['property_value'], row['property_value_gbp_rate'], row['other_assets'], row['other_assets_gbp_rate'], row['total_liabilities_in_base_currency'], row['default_gbp_rate'], row['average_3_month_salary'], row['average_3_month_salary_gbp_rate']), axis=1)

```
Timer unit: 1e-06 s

Total time: 0.025267 s
File: <ipython-input-62-f9033797a7f1>
Function: net_asset_salary_ratio at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def net_asset_salary_ratio(prop, prop_rate, assets, assets_rate, liabilities, liabilities_rate, salary, salary_rate):
     2     20000        15930      0.8     63.0      net_position = prop*prop_rate + assets*assets_rate - liabilities*liabilities_rate
     3     20000         9337      0.5     37.0      return net_position / (salary*salary_rate)
```

In [60]:
%lprun -f net_asset_salary_ratio asset_to_salary = net_asset_salary_ratio(df['property_value'], df['property_value_gbp_rate'], df['other_assets'], df['other_assets_gbp_rate'],df['total_liabilities_in_base_currency'], df['default_gbp_rate'], df['average_3_month_salary'],df['average_3_month_salary_gbp_rate'])

```
Timer unit: 1e-06 s

Total time: 0.003366 s
File: <ipython-input-62-f9033797a7f1>
Function: net_asset_salary_ratio at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def net_asset_salary_ratio(prop, prop_rate, assets, assets_rate, liabilities, liabilities_rate, salary, salary_rate):
     2         1         2417   2417.0     71.8      net_position = prop*prop_rate + assets*assets_rate - liabilities*liabilities_rate
     3         1          949    949.0     28.2      return net_position / (salary*salary_rate)
```

In [61]:
%lprun -f net_asset_salary_ratio asset_to_salary = net_asset_salary_ratio(df['property_value'].values, df['property_value_gbp_rate'].values,df['other_assets'].values, df['other_assets_gbp_rate'].values, df['total_liabilities_in_base_currency'].values,df['default_gbp_rate'].values, df['average_3_month_salary'].values,df['average_3_month_salary_gbp_rate'].values)

```
Timer unit: 1e-06 s

Total time: 0.000514 s
File: <ipython-input-62-f9033797a7f1>
Function: net_asset_salary_ratio at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def net_asset_salary_ratio(prop, prop_rate, assets, assets_rate, liabilities, liabilities_rate, salary, salary_rate):
     2         1          437    437.0     85.0      net_position = prop*prop_rate + assets*assets_rate - liabilities*liabilities_rate
     3         1           77     77.0     15.0      return net_position / (salary*salary_rate)
```

---
<h2>Example 6: If</h2>

In [62]:
%timeit thresholded = df['average_3_month_salary'].apply(lambda x: np.nan if x < 2000 else x).values
%timeit thresholded = df['average_3_month_salary'].where(df['average_3_month_salary'] >= 2000).values
%timeit thresholded = np.where(df['average_3_month_salary'].values >= 2000, df['average_3_month_salary'].values, np.nan)

7.54 ms ± 3.82 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
728 µs ± 111 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:257: RuntimeWarning: invalid value encountered in greater_equal


107 µs ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


---
<h1>Take homes</h1>


<h2> 1) NumPy! </h2>
<br>
<h2> 2) Use Pandas builtins </h2>
<br>
<h2> 3) Vectorise! </h2>
<br>
<h2> 4) iteritems / iterrows is for special occassions!
 - Avoid loops, but where you absolutely have to use loops, use .apply </h2>
<br>
<h2> 5) Use performance optimisers (numexpr, bottleneck) </h2>
<br>
<h2> 6) Look under the hood! 
 - Covenient profiling tools </h2>
<br>
<h2> 7) If you need, more sophisticated optimisation mechanisms do exist (cython, numba) </h2>

<h1><center>Using Pandas thoughtfully can give 100x or more speed ups!</center></h1>

---
<h2>Resources</h2>

Pandas and Dask from the Inside, Stephen Simmons <br>
Berlin, PyData 2017 <br>
https://github.com/stevesimmons/pydata-berlin2017-pandas-and-dask-from-the-inside <br>
https://www.youtube.com/watch?v=YGk09nK_xnM

<br>
No More Sad Pandas: Optimizing Pandas Code for Speed and Efficiency, Sofia Heisler <br>
Portland Oregon, PyCon 2017 <br>
https://github.com/sversh/pycon2017-optimizing-pandas <br>
https://www.youtube.com/watch?v=HN5d490_KKk <br>
https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

<br>
Performance Pandas, Jeff Reback <br>
London, PyData 2015 <br>
https://www.youtube.com/watch?v=xUBoPK6FGIU
https://github.com/jreback/pydata2015-london